In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

In [ ]:
ziped=tf.keras.utils.get_file(
    origin="https://storage.googleapis.com/tensorflow/tf-keras-datasets/jena_climate_2009_2016.csv.zip",
    fname="jena_climate_2009_2016.csv.zip",
    extract=True)
csv_path, _ = os.path.splitext(ziped)

In [ ]:
df = pd.read_csv(csv_path)
df

,Date Time,p (mbar),T (degC),Tpot (K),Tdew (degC),rh (%),VPmax (mbar),VPact (mbar),VPdef (mbar),sh (g/kg),H2OC (mmol/mol),rho (g/m**3),wv (m/s),max. wv (m/s),wd (deg)
0,01.01.2009 00:10:00,996.52,-8.02,265.40,-8.90,93.30,3.33,3.11,0.22,1.94,3.12,1307.75,1.03,1.75,152.3
1,01.01.2009 00:20:00,996.57,-8.41,265.01,-9.28,93.40,3.23,3.02,0.21,1.89,3.03,1309.80,0.72,1.50,136.1
2,01.01.2009 00:30:00,996.53,-8.51,264.91,-9.31,93.90,3.21,3.01,0.20,1.88,3.02,1310.24,0.19,0.63,171.6
3,01.01.2009 00:40:00,996.51,-8.31,265.12,-9.07,94.20,3.26,3.07,0.19,1.92,3.08,1309.19,0.34,0.50,198.0
4,01.01.2009 00:50:00,996.51,-8.27,265.15,-9.04,94.10,3.27,3.08,0.19,1.92,3.09,1309.00,0.32,0.63,214.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
420546,31.12.2016 23:20:00,1000.07,-4.05,269.10,-8.13,73.10,4.52,3.30,1.22,2.06,3.30,1292.98,0.67,1.52,240.0
420547,31.12.2016 23:30:00,999.93,-3.35,269.81,-8.06,69.71,4.77,3.32,1.44,2.07,3.32,1289.44,1.14,1.92,234.3
420548,31.12.2016 23:40:00,999.82,-3.16,270.01,-8.21,67.91,4.84,3.28,1.55,2.05,3.28,1288.39,1.08,2.00,215.2
420549,31.12.2016 23:50:00,999.81,-4.23,268.94,-8.53,71.80,4.46,3.20,1.26,1.99,3.20,1293.56,1.49,2.16,225.8


In [ ]:
df.index=df["Date Time"]
df.index = pd.to_datetime(df['Date Time'], format='%d.%m.%Y %H:%M:%S')

In [ ]:
temp = df['T (degC)']

In [ ]:
def dataset(window_size,data):
  data=data.to_numpy()
  x=[]
  y=[]
  for i in range(len(data)-window_size):
    row = [[a] for a in data[i:i+window_size]]
    x.append(row)
    label = data[i+window_size]
    y.append(label)
  return np.array(x), np.array(y)

In [ ]:
X1,y1=dataset(5,temp)

In [ ]:
from tensorflow.keras.layers import Dense,LSTM,Input,Flatten
from tensorflow.keras.models import Sequential
import tensorflow.keras.models

In [ ]:
X_train1, y_train1 = X1[:60000], y1[:60000]
X_val1, y_val1 = X1[60000:65000], y1[60000:65000]
X_test1, y_test1 = X1[65000:], y1[65000:]
X_train1.shape, y_train1.shape, X_val1.shape, y_val1.shape, X_test1.shape, y_test1.shape

((60000, 5, 1), (60000,), (5000, 5, 1), (5000,), (355546, 5, 1), (355546,))

In [ ]:
model=Sequential([
    Dense(5,input_shape=(5,1)),
    LSTM(5,activation="relu"),
    Dense(1,activation="sigmoid")

])

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.optimizers import Adam

In [ ]:
cp1 = ModelCheckpoint('model/', save_best_only=True)
model.compile(loss=MeanSquaredError(), optimizer=Adam(learning_rate=0.0001), metrics=[RootMeanSquaredError()])

In [ ]:
model.fit(X_train1, y_train1, validation_data=(X_val1, y_val1), epochs=10, callbacks=[cp1])

Epoch 1/10
1875/1875 [==============================] - 11s 5ms/step - loss: 123.7364 - root_mean_squared_error: 11.1237 - val_loss: 41.8874 - val_root_mean_squared_error: 6.4721
Epoch 2/10
1875/1875 [==============================] - 9s 5ms/step - loss: 119.5610 - root_mean_squared_error: 10.9344 - val_loss: 41.6074 - val_root_mean_squared_error: 6.4504
Epoch 3/10
1875/1875 [==============================] - 10s 5ms/step - loss: 119.4844 - root_mean_squared_error: 10.9309 - val_loss: 41.5360 - val_root_mean_squared_error: 6.4448
Epoch 4/10
1875/1875 [==============================] - 10s 5ms/step - loss: 119.4629 - root_mean_squared_error: 10.9299 - val_loss: 41.5143 - val_root_mean_squared_error: 6.4432
Epoch 5/10
1875/1875 [==============================] - 11s 6ms/step - loss: 119.4563 - root_mean_squared_error: 10.9296 - val_loss: 41.5078 - val_root_mean_squared_error: 6.4427
Epoch 6/10
1875/1875 [==============================] - 10s 6ms/step - loss: 119.4542 - root_mean_squared_